
---

## 🌐 Form Validation Overview

* Validation ensures **required fields are filled** and input is in the **correct format**.
* Can be done:

  1. **Manually** in Flask (simple checks)
  2. **Using Flask-WTF** (more structured and automatic)

Here we’ll cover **basic manual validation** first.

---

## ✅ 1️⃣ Manual Validation Example

**Folder structure:**

```
project/
│   app.py
└───templates/
    │   register.html
```

**app.py**

```python
from flask import Flask, render_template, request

app = Flask(__name__)

@app.route("/register", methods=["GET", "POST"])
def register():
    error = ""
    if request.method == "POST":
        username = request.form.get("username")
        password = request.form.get("password")
        email = request.form.get("email")

        # Basic validation
        if not username or not password or not email:
            error = "All fields are required!"
        elif len(username) < 3:
            error = "Username must be at least 3 characters long."
        elif "@" not in email:
            error = "Enter a valid email address."
        else:
            return f"<h1>Registration Successful!</h1><p>Welcome, {username}!</p>"

    return render_template("register.html", error=error)
```

**templates/register.html**

```html
<!DOCTYPE html>
<html>
<head>
    <title>Register</title>
</head>
<body>
    <h1>Register</h1>
    {% if error %}
        <p style="color:red">{{ error }}</p>
    {% endif %}

    <form method="POST">
        Username: <input type="text" name="username" required><br><br>
        Email: <input type="email" name="email" required><br><br>
        Password: <input type="password" name="password" required><br><br>
        <input type="submit" value="Register">
    </form>
</body>
</html>
```

---

### 🔎 How It Works

1. **Check for empty fields** → `if not username ...`
2. **Check for specific conditions** → e.g., minimum length or valid email
3. **Display error messages** dynamically using `{{ error }}` in template
4. Only **process the form** if all validations pass

---

## ✅ 2️⃣ Common Basic Validations

| Validation     | Example                |
| -------------- | ---------------------- |
| Required field | `if not username`      |
| Minimum length | `if len(password) < 6` |
| Email format   | `if "@" not in email`  |
| Numeric input  | `if not age.isdigit()` |

---

## ✅ 3️⃣ Next Steps

* For **more advanced validation**, you can use **Flask-WTF** which automatically handles:

  * Required fields
  * Length, email, regex
  * CSRF protection
  * Cleaner code using **Form classes**

---

This approach is great for **learning and simple apps**, while Flask-WTF is better for **production-level apps**.

---





---

## 🌐 What is Flask-WTF (Flask – Web Forms Toolkit)?

* **Flask-WTF** is a Flask extension that integrates **WTForms**.
* Provides:

  1. **Form classes** for clean code
  2. **Built-in validation** (required fields, email, length, regex, etc.)
  3. **CSRF(Cross-Site Request Forgery) protection** automatically
  4. **Reusable forms** across multiple templates

---


**Cross-Site Request Forgery (CSRF)** ✅

---

### **Explanation:**

* **Cross-Site** → The attack originates from a different website than the one you are logged into.
* **Request Forgery** → The attacker tricks your browser into making an unwanted request without your consent.

---

### **How it Works (Simplified)**

1. You are logged into a website (e.g., online banking).
2. You visit a malicious site in another tab.
3. That site secretly sends a request (like transferring money) to the bank **using your logged-in session**.
4. The bank thinks it’s a legitimate request from you → **action happens without your consent**.

---

### **CSRF Protection in Flask-WTF**

* Flask-WTF automatically adds a **CSRF token** to forms.
* The server validates the token on form submission.
* If the token is missing or invalid → request is rejected.

---



## ✅ 1️⃣ Installation

```bash
pip install Flask-WTF
```

---

## ✅ 2️⃣ Basic Flask-WTF Form Example

**Folder structure:**

```
project/
│   app.py
└───templates/
    │   register.html
```

**app.py**

```python
from flask import Flask, render_template, flash
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField
from wtforms.validators import DataRequired, Email, Length

app = Flask(__name__)
app.secret_key = "mysecretkey"  # Required for CSRF protection

# 1️⃣ Define the Form class
class RegistrationForm(FlaskForm):
    username = StringField("Username", validators=[DataRequired(), Length(min=3, max=20)])
    email = StringField("Email", validators=[DataRequired(), Email()])
    password = PasswordField("Password", validators=[DataRequired(), Length(min=6)])
    submit = SubmitField("Register")

# 2️⃣ Route for form
@app.route("/register", methods=["GET", "POST"])
def register():
    form = RegistrationForm()
    if form.validate_on_submit():  # Validates all fields
        username = form.username.data
        email = form.email.data
        password = form.password.data
        flash(f"Account created for {username}!", "success")
        # Here you can save data to database
        return render_template("register.html", form=form)
    return render_template("register.html", form=form)

if __name__ == "__main__":
    app.run(debug=True)
```

---

**templates/register.html**

```html
<!DOCTYPE html>
<html>
<head>
    <title>Registration Form</title>
</head>
<body>
    <h1>Register</h1>
    <form method="POST">
        {{ form.hidden_tag() }}  <!-- CSRF token -->

        <p>
            {{ form.username.label }}<br>
            {{ form.username(size=32) }}<br>
            {% for error in form.username.errors %}
                <span style="color:red">{{ error }}</span>
            {% endfor %}
        </p>

        <p>
            {{ form.email.label }}<br>
            {{ form.email(size=32) }}<br>
            {% for error in form.email.errors %}
                <span style="color:red">{{ error }}</span>
            {% endfor %}
        </p>

        <p>
            {{ form.password.label }}<br>
            {{ form.password(size=32) }}<br>
            {% for error in form.password.errors %}
                <span style="color:red">{{ error }}</span>
            {% endfor %}
        </p>

        <p>{{ form.submit() }}</p>
    </form>

    {% with messages = get_flashed_messages(with_categories=true) %}
        {% if messages %}
            {% for category, message in messages %}
                <p style="color:green">{{ message }}</p>
            {% endfor %}
        {% endif %}
    {% endwith %}
</body>
</html>
```

---

## 🔎 How It Works

1. **Form Class** → `RegistrationForm` defines fields and validators.
2. **CSRF Protection** → `hidden_tag()` adds CSRF token automatically.
3. **Validation** → `form.validate_on_submit()` checks all validators before processing.
4. **Flashing messages** → `flash()` shows success messages after submission.
5. **Reusable and clean templates** → form fields can be reused in multiple pages.

---

## ✅ Advantages of Flask-WTF

| Feature             | Plain HTML Form | Flask-WTF                              |
| ------------------- | --------------- | -------------------------------------- |
| Validation          | Manual          | Built-in (DataRequired, Email, Length) |
| CSRF                | Manual          | Automatic                              |
| Reusable            | Low             | High (Form classes)                    |
| Cleaner Python code | Less structured | Form classes handle logic              |

---

